In [ ]:
import numpy as np
import pandas as pd
from flask import Flask, request, render_template
import joblib

app= Flask(__name__)
model = joblib.load('model_Credit.pkl')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    form_data = request.form.to_dict(flat=False)
    features = []

    # Standard inputs
    standard_inputs = [
        'Monthly_Inhand_Salary',
        'Num_Credit_Card',
        'Interest_Rate',
        'Delay_from_due_date',
        'Num_of_Delayed_Payment',
        'Changed_Credit_Limit',
        'Num_Credit_Inquiries',
        'Credit_Mix',
        'Outstanding_Debt',
        'Credit_Utilization_Ratio',
        'Credit_History_Age_Months',
        'Payment_of_Min_Amount',
        'Amount_invested_monthly',
        'Payment_Behaviour',
        'Monthly_Balance'
    ]

    for key in standard_inputs:
        features.append(float(form_data.get(key, [0])[0]))

    loan_types = [
        'Credit-Builder Loan',
        'Not Specified',
        'No Loan',
        'Personal Loan',
        'Debt Consolidation Loan',
        'Auto Loan',
        'Payday Loan',
        'Student Loan',
        'Home Equity Loan',
        'Mortgage Loan'
    ]
    loan_values = [0] * len(loan_types)


    if 'loan_type[]' in form_data and 'num_loan[]' in form_data:
        loan_types_list = form_data['loan_type[]']
        loan_amounts_list = form_data['num_loan[]']

        for loan_type, loan_amount in zip(loan_types_list, loan_amounts_list):
            if loan_type in loan_types:
                try:
                    loan_values[loan_types.index(loan_type)] = float(loan_amount)
                except ValueError:
                    loan_values[loan_types.index(loan_type)] = 0 


    if all(value == 0 for value in loan_values):
        loan_values[loan_types.index('No Loan')] = 1

    features.extend(loan_values)


    feature_columns = standard_inputs + loan_types
    features_df = pd.DataFrame([features], columns=feature_columns)

    prediction = model.predict(features_df)

    prediction_labels = ["Poor", "Standard", "Good"]
    prediction_label = prediction_labels[int(prediction[0])]

    return render_template(
        'index.html',
        prediction_result=f"Predicted Credit Score: {prediction_label}"
    )


if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Jun/2025 13:47:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2025 13:47:15] "GET /favicon.ico HTTP/1.1" 404 -
